**분석개요**

    A. Wallpaper 로그인 전환 이탈 리포트
        1. KPI
            - PV
            - 다운로드
            - 전환율

In [1]:
# basic
import gc
import os
import sys
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--------------------#
# handling
#--------------------#
import math
import time
import random
# import openpyxl
import importlib
import xlsxwriter
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime, timedelta
from scipy.stats import pearsonr
import difflib

#--------------------#
# Vis
#--------------------#
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
## Vis for jupyter theme
from IPython.display import Image
from jupyterthemes import jtplot
jtplot.style(theme= 'grade3', context='notebook', ticks=True, grid=False) ## dark backgroud jupyter notebook theme
# plt.style.use(['seaborn-white']) ## dark background style ## white style : 'seaborn-white'
plt.style.use(['dark_background']) ## dark background style ## white style : 'seaborn-white'
## Vis for korean 
import matplotlib.font_manager as fm
font_location = '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf'
fprop = fm.FontProperties(fname=font_location)
font_name = fprop.get_name()
matplotlib.rc('font', family=font_name)

In [2]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import kto_prep
from kto_config import *
from kto_prep import *
from kto_util import *

for pkg in [kto_config, kto_prep, kto_util] :
    _ = importlib.reload(pkg)

#-------------------#
# pickle
#-------------------#
sys.path.append("/home/minkyung62/analysis_report/00_custom_analysis/kto_custom/notebook_git/return_pickle.py")
import return_pickle
from return_pickle import *
for pkg in [return_pickle] :
    _ = importlib.reload(pkg)



sys.path.append("/home/das_share/analysis/cdj/src/SphereCDJ.py")
import SphereCDJ
from SphereCDJ import *
for pkg in [SphereCDJ] :
    _ = importlib.reload(pkg)

# 1.Data Import

In [3]:
#------------------------------------------------#
# 1. log Data import
#------------------------------------------------#
## target period
s_date, e_date = '20221020', '20221130' 
today = datetime.strptime(e_date, '%Y%m%d') # today = datetime(2021,5,6)
dates = (datetime.strptime(e_date, '%Y%m%d') - datetime.strptime(s_date, '%Y%m%d')).days + 1 
change_date = datetime(2022,11,3)    

# CheckDate.check_date_return_pickle(e_date, s_date = s_date)

df_app_log = ReadFile.read_pickle(e_date=e_date, s_date = s_date)
# ## checking
# df_app_log_before = ReadFile.read_pickle(e_date='20221102', s_date=s_date)
# df_app_log_after = ReadFile.read_pickle(e_date=e_date, s_date='20221103')

In [5]:
df_app_log[KEY_ID_USER].isna().sum()

0

# 2.Preprocess

In [6]:
#------------------------------------------------#
# 1. prep
#------------------------------------------------#
## 1) log
df_app_log = SpherePrep.BasicPrep.basic_prep(df_app_log)                # log data

## 2) user prop
df_prop = SpherePrep.Prop.df_prop_pipe(df_app_log, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER], drop_none_prop=False)       # user data

## 3) param - 1
# _, df_param_all = DataImport.json_to_dataframe_nodeN(
#     df_app_log, [KEY_ID_DEVICE, KEY_ID_USER]
#     )

#------------------------------------------------#
# 2. param detail data
#------------------------------------------------#
## 1) wallpaper 관련 param data 생성
EVENT_MAIN_WALLPAPER = 'wallPaperMain'
EVENT_DOWNLOAD_WALLPAPER = 'wallPaperDownload'

df_app_log_wallpaper = df_app_log[df_app_log['abs_events'].apply(lambda x: True if EVENT_MAIN_WALLPAPER in x else False)]
df_app_log_download = df_app_log[df_app_log['abs_events'].apply(lambda x: True if EVENT_DOWNLOAD_WALLPAPER in x else False)]

## 2) datetime으로 type 변환
df_app_log_wallpaper['day'] = pd.to_datetime(df_app_log_wallpaper['day'], format = '%Y-%m-%d')
df_app_log_download['day'] = pd.to_datetime(df_app_log_download['day'], format = '%Y-%m-%d')

##### debugging

In [7]:
df_app_log[KEY_ID_USER].isna().sum()

3925732

In [10]:
## checking
df_app_log_after = ReadFile.read_pickle(e_date=e_date, s_date='20221103')
df_app_log_after = SpherePrep.BasicPrep.basic_prep(df_app_log_after)  

df_app_log_after[KEY_ID_USER].isna().sum()

0

In [11]:
df_app_log[df_app_log['date']>='20221103'][KEY_ID_USER].isna().sum()

2767143

## 1)Utils

In [8]:
def date_setting(date):
    _date = datetime.strptime(date, "%Y-%m-%d")
    return _date

def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y%m%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates


def return_df(_df_target1, _df_target2, end_date, start_date = None):
    ## 특정 주차 데이터 구하기
    if start_date != None:
        _df_target1 = _df_target1[_df_target1['day'] >= start_date]
        _df_target2 = _df_target2[_df_target2['day'] >= start_date]

    _df_output1 = _df_target1[_df_target1['day'] < end_date]
    _df_output2 = _df_target2[_df_target2['day'] < end_date]


    return _df_output1, _df_output2

def calculate_kpi(df_pv, df_download, text):
    dict_kpi= {}

    ## 1) 주차별 KPI 
    df_pv['pv_cnt'] = df_pv['abs_events'].apply(
                        lambda x : Counter(x)[EVENT_MAIN_WALLPAPER] 
                        if EVENT_MAIN_WALLPAPER in x else 0)
    dict_kpi['pv_cnt'] = df_pv['pv_cnt'].sum()

    dict_kpi['pv_device_cnt'] = df_pv[KEY_ID_DEVICE].nunique()

    df_download['download_cnt'] = df_download['abs_events'].apply(
                                    lambda x: Counter(x)[EVENT_DOWNLOAD_WALLPAPER] 
                                    if EVENT_DOWNLOAD_WALLPAPER in x else 0)
    dict_kpi['download_cnt']  = df_download['download_cnt'].sum()

    dict_kpi['dw_device_cnt'] = df_download[KEY_ID_DEVICE].nunique()

    if (dict_kpi['pv_cnt'] != 0 and dict_kpi['pv_device_cnt']!= 0):
        dict_kpi['PV 대비 다운로드 전환율']  = dict_kpi['download_cnt'] / dict_kpi['pv_cnt']
        dict_kpi['PV 기기당 다운로드 수']  = dict_kpi['download_cnt'] / dict_kpi['pv_device_cnt']
        dict_kpi['기기당 다운로드 전환율'] = dict_kpi['dw_device_cnt'] / dict_kpi['pv_device_cnt']

    else:
        dict_kpi['PV 대비 다운로드 전환율']  = 0
        dict_kpi['PV 기기당 다운로드 수']  = 0
        dict_kpi['기기당 다운로드 전환율'] = 0
    
    df_week_kpi = pd.DataFrame.from_dict([dict_kpi]).rename(index={0:text})

    ## 2) 일별 KPI
    _df_output_1_1 = df_pv.groupby('day')['pv_cnt'].sum().to_frame(name = 'pv_cnt')
    _df_output_1_2 = df_download.groupby('day')['download_cnt'].sum().to_frame(name = 'download_cnt')

    _df_output_1_3 = df_pv.groupby('day')[KEY_ID_DEVICE].nunique().to_frame(name = 'pv_device_cnt')
    _df_output_1_4 = df_download.groupby('day')[KEY_ID_DEVICE].nunique().to_frame(name = 'dw_device_cnt')

    df_daily_kpi = pd.concat([_df_output_1_1, _df_output_1_2, _df_output_1_3, _df_output_1_4], axis = 1)

    df_daily_kpi['PV 대비 다운로드 전환율'] = df_daily_kpi['download_cnt'] / df_daily_kpi['pv_cnt']
    df_daily_kpi['PV 기기당 다운로드 수'] = df_daily_kpi['download_cnt'] / df_daily_kpi['pv_device_cnt']
    df_daily_kpi['기기당 다운로드 전환율'] = df_daily_kpi['dw_device_cnt'] / df_daily_kpi['pv_device_cnt']

    return df_week_kpi, df_daily_kpi

# 3.Analysis

## 1)KPI
- PV
- 다운로드
- 전환율
    - 페이지뷰 대비
    - 사용자 대비

In [6]:
#########################################
# 0. 전체 KPI
## ** 서비스 전환 전후 평균 KPI 도출
#########################################
## 1) 전환 전
_date = date_setting("2022-11-03")

df_event_wallpaper_before, df_event_download_before =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date)

df_event_wallpaper_after, df_event_download_after =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date= date_setting("2022-12-01"), start_date=_date)

df_output_before, _ =\
    calculate_kpi(df_event_wallpaper_before, df_event_download_before, text = '전환 전')

df_output_after, _ =\
    calculate_kpi(df_event_wallpaper_after, df_event_download_after, text = '전환 후')

df_output_total = pd.concat([df_output_before, df_output_after], axis=0)
df_output_total

,pv_cnt,pv_device_cnt,download_cnt,dw_device_cnt,PV 대비 다운로드 전환율,PV 기기당 다운로드 수,기기당 다운로드 전환율
전환 전,3249,2283,4271,800,1.314558,1.870784,0.350416
전환 후,5433,2887,3315,428,0.610160,1.148251,0.148251


In [7]:
#########################################
# 1. 주차별/일별 KPI
## ** 한 주 시작일 = 월요일 -> 태깅이 월요일에 반영되어 일요일 데이터가 없어서 월요일로 시작점을 설정함
#########################################
## date setting
_date_1 = date_setting("2022-10-31")
_date_2 = date_setting("2022-11-07")
_date_3 = date_setting("2022-11-14")
_date_4 = date_setting("2022-11-21")
_date_5 = date_setting("2022-11-28")
_date_6 = date_setting("2022-12-01")


# ## data setting
_df_event_wallpaper_1, _df_event_download_1 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_1)

_df_event_wallpaper_2, _df_event_download_2 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_2, start_date=_date_1)

_df_event_wallpaper_3,_df_event_download_3 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_3, start_date=_date_2)

_df_event_wallpaper_4,_df_event_download_4 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_4, start_date=_date_3)

_df_event_wallpaper_5,_df_event_download_5 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_5, start_date=_date_4)

_df_event_wallpaper_6,_df_event_download_6 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_6, start_date=_date_5)

## calculate KPI
lst_output_1 = [_df_event_wallpaper_1, _df_event_wallpaper_2, _df_event_wallpaper_3, _df_event_wallpaper_4, _df_event_wallpaper_5, _df_event_wallpaper_6]
lst_output_2 = [_df_event_download_1, _df_event_download_2, _df_event_download_3, _df_event_download_4, _df_event_download_5, _df_event_download_6]
lst_text = ['10월_4주차', '11월_1주차', '11월_2주차', '11월_3주차', '11월_4주차', '11월_5주차']

df_output_week = pd.DataFrame()
df_output_daily = pd.DataFrame()

for _df1, _df2, _t in zip(lst_output_1, lst_output_2, lst_text):
    _df_output1, _df_output2 =\
        calculate_kpi(_df1, _df2, text = _t)
    
    df_output_week = pd.concat([df_output_week, _df_output1])
    df_output_daily = pd.concat([df_output_daily, _df_output2])

df_output_week
df_output_daily

,pv_cnt,pv_device_cnt,download_cnt,dw_device_cnt,PV 대비 다운로드 전환율,PV 기기당 다운로드 수,기기당 다운로드 전환율
10월_4주차,1984,1455,2310,461,1.164315,1.587629,0.316838
11월_1주차,2261,1460,2399,443,1.061035,1.643151,0.303425
11월_2주차,1458,862,979,114,0.671468,1.135731,0.132251
11월_3주차,1238,719,682,95,0.550889,0.948540,0.132128
11월_4주차,1121,635,642,84,0.572703,1.011024,0.132283
11월_5주차,620,332,574,70,0.925806,1.728916,0.210843


,pv_cnt,download_cnt,pv_device_cnt,dw_device_cnt,PV 대비 다운로드 전환율,PV 기기당 다운로드 수,기기당 다운로드 전환율
day,,,,,,,
2022-10-24,115,77,65,13,0.669565,1.184615,0.200000
2022-10-25,149,190,96,31,1.275168,1.979167,0.322917
2022-10-26,117,155,90,30,1.324786,1.722222,0.333333
2022-10-27,96,99,85,19,1.031250,1.164706,0.223529
2022-10-28,291,371,237,82,1.274914,1.565401,0.345992
2022-10-29,763,944,578,177,1.237221,1.633218,0.306228
2022-10-30,453,474,356,118,1.046358,1.331461,0.331461
2022-10-31,324,376,257,92,1.160494,1.463035,0.357977
2022-11-01,666,1390,455,192,2.087087,3.054945,0.421978


In [18]:
## 일평균 다운로드 완료 기기 수 추출
_, _df_event_download_before =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date = date_setting("2022-11-03"))

_, _df_event_download_after =\
    return_df(df_app_log_wallpaper, df_app_log_download, start_date = date_setting("2022-11-03"),\
        end_date = date_setting("2022-12-01"))

_df_event_download_before.groupby('day')[KEY_ID_DEVICE].nunique().mean()
_df_event_download_after.groupby('day')[KEY_ID_DEVICE].nunique().mean()

82.0

17.142857142857142

In [8]:
#########################################
# 1. 주차별/일별 KPI(ver2)
## ** 한 주 시작일 = 월요일 -> 태깅이 월요일에 반영되어 일요일 데이터가 없어서 월요일로 시작점을 설정함
#########################################
## date setting
_date_1 = date_setting("2022-10-31")
_date_2 = date_setting("2022-11-07")
_date_3 = date_setting("2022-11-14")
_date_4 = date_setting("2022-11-21")
_date_5 = date_setting("2022-11-28")
_date_6 = date_setting("2022-12-01")


# ## data setting
_df_event_wallpaper_12, _df_event_download_12 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_1)

_df_event_wallpaper_22, _df_event_download_22 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=date_setting("2022-11-03"), start_date=_date_1)

_df_event_wallpaper_32,_df_event_download_32 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_2, start_date=date_setting("2022-11-03"))

_df_event_wallpaper_42,_df_event_download_42 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_3, start_date=_date_2)

_df_event_wallpaper_52,_df_event_download_52 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_4, start_date=_date_3)

_df_event_wallpaper_62,_df_event_download_62 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_5, start_date=_date_4)

_df_event_wallpaper_72,_df_event_download_72 =\
    return_df(df_app_log_wallpaper, df_app_log_download, end_date=_date_6, start_date=_date_5)

## calculate KPI
lst_output_12 = [
    _df_event_wallpaper_12, _df_event_wallpaper_22, _df_event_wallpaper_32, 
    _df_event_wallpaper_42, _df_event_wallpaper_52, _df_event_wallpaper_62, _df_event_wallpaper_72]
lst_output_22 = [
    _df_event_download_12, _df_event_download_22, _df_event_download_32, 
    _df_event_download_42, _df_event_download_52, _df_event_download_62, _df_event_download_72]
lst_text2 = ['10월_4주차', '11월_1주차(10/31~11/02)', '11월_1주차(11/03~11/06)', '11월_2주차', '11월_3주차', '11월_4주차', '11월_5주차']

df_output_week2 = pd.DataFrame()
# df_output_daily2 = pd.DataFrame()

for _df1, _df2, _t in zip(lst_output_12, lst_output_22, lst_text2):
    _df_output1, _df_output2 =\
        calculate_kpi(_df1, _df2, text = _t)
    
    df_output_week2 = pd.concat([df_output_week2, _df_output1])
    # df_output_daily2 = pd.concat([df_output_daily2, _df_output2])

df_output_week2
# df_output_daily2

,pv_cnt,pv_device_cnt,download_cnt,dw_device_cnt,PV 대비 다운로드 전환율,PV 기기당 다운로드 수,기기당 다운로드 전환율
10월_4주차,1984,1455,2310,461,1.164315,1.587629,0.316838
11월_1주차(10/31~11/02),1265,882,1961,347,1.550198,2.223356,0.393424
11월_1주차(11/03~11/06),996,627,438,100,0.439759,0.698565,0.159490
11월_2주차,1458,862,979,114,0.671468,1.135731,0.132251
11월_3주차,1238,719,682,95,0.550889,0.948540,0.132128
11월_4주차,1121,635,642,84,0.572703,1.011024,0.132283
11월_5주차,620,332,574,70,0.925806,1.728916,0.210843


## 2)2차피드백
- 신규 회원가입 비율

In [11]:
#########################################
# 2. 월페이퍼 신규 회원가입 비율
#########################################
## 0) 과거 데이터 생성
def import_prop_manual_data(file_path) :
    df_prop_manual = pd.read_excel(file_path, engine='openpyxl') 
    df_prop_manual = df_prop_manual.rename(
        columns = {
            [x for x in df_prop_manual.columns if '회원ID' in x][0] : KEY_ID_USER,
            [x for x in df_prop_manual.columns if '성별' in x][0] : 'gender',
            [x for x in df_prop_manual.columns if '생년' in x][0] : 'birth_y',
            [x for x in df_prop_manual.columns if '회원가입일시' in x][0] : 'created_date_user',
            }
        )
    lst_female_label = ['W', 'w', '여자', '여성', '여']
    lst_male_label = ['M', 'm', '남자', '남성', '남']
    df_prop_manual['gender'] =\
        df_prop_manual['gender'].apply(lambda x : 'f' if x in lst_female_label else 'm')

    return df_prop_manual

manual_prop_file_path = '/home/minkyung62/analysis_report/00_custom_analysis/kto_custom/data/in/KTO_user_prop_221108.xlsx'
df_prop_manual = import_prop_manual_data(manual_prop_file_path)
df_prop_manual

## 1) 과거 미회원가입자 추출
### 과거 회원가입을 진행한 유저 -> 제거 대상
lst_user_df_prop_manual = list(df_prop_manual['user_id'].unique())
lst_user_login_past = []

for i in range(len(df_prop_manual)):
    if df_prop_manual.iloc[i]['created_date_user'] < datetime.strptime('2022-11-03 00:00:00', '%Y-%m-%d %H:%M:%S'):
        lst_user_login_past.append(df_prop_manual.iloc[i]['user_id'])

lst_user_login_past = list(set(lst_user_login_past))                ## 과거 회원가입한 유저(11/02일까지)

,user_id,유형,gender,birth_y,SSO회원ID,created_date_user,마지막로그인일시
0,408dbc94-2190-40ba-a9b3-056d800c0ccc,네이버,m,1992.0,NV00000000000000000000000000000000000034092865...,2022-11-08 14:17:24,2022-11-08 14:17:24
1,adfdfab2-a232-4598-962c-2d18bafccc08,카카오톡,f,1977.0,KA00000000000000000000000000000000001674768525...,2022-11-08 14:10:23,2022-11-08 14:10:23
2,98e25352-16c0-4ca1-b9aa-ff775472b2e6,카카오톡,m,1973.0,KA00000000000000000000000000000000002516624806...,2022-11-08 14:08:02,2022-11-08 14:08:02
3,d376d9d4-8dfa-4499-94aa-cc67123bb25d,카카오톡,f,1981.0,KA00000000000000000000000000000000002516617858...,2022-11-08 14:02:44,2022-11-08 14:02:44
4,2c8c94f0-13b3-4280-8a31-8d3b6607583d,네이버,f,1974.0,NV00000000000000000000000000000000000041747703...,2022-11-08 13:43:29,2022-11-08 13:43:29
...,...,...,...,...,...,...,...
325580,764582cc-838c-403c-8036-53e9c8151839,카카오톡,m,NaN,NaN,NaT,NaT
325581,8fac9593-c51f-41ef-a854-55848f33d3a8,카카오톡,m,NaN,NaN,NaT,NaT
325582,9473f8fa-471d-4022-9d18-1f1926176611,카카오톡,m,NaN,NaN,NaT,NaT
325583,e03ebcca-18d8-4f6e-a9ec-dc3edc4484ce,네이버,m,NaN,NaN,NaT,NaT


In [12]:
#########################################
# 2. 월페이퍼 신규 회원가입 비율
#########################################
## 2) Y = 과거 미회원가입(=비로그인) & 3일 이후 월페이퍼 접속
### (1) 과거 미회원가입 기기 id 추출
lst_device_login_past = list(df_app_log[KEY_ID_DEVICE][df_app_log[KEY_ID_USER].isin(lst_user_login_past)])
lst_device_loginx_past = list(
                        set(list(df_app_log[KEY_ID_DEVICE].unique())) - set(lst_device_login_past)
                        )

### (2) 3일 이후 월페이퍼 접속자 
df_app_log_wallpaper_after = df_app_log_wallpaper[df_app_log_wallpaper['day'] > '20221102']
lst_wallpaper_after = list(df_app_log_wallpaper_after[KEY_ID_DEVICE].unique())

### (3) 과거 미회원가입(=비로그인) & 3일 이후 월페이퍼 접속 기기 id 추출
lst_y1 = list(set(lst_device_loginx_past) & set(lst_wallpaper_after))
print('y: ', len(lst_y1))

#########################################################################################

# 3) X = 과거 비로그인 & 3일 이후 월페이퍼 접속 o & 3일 이후 로그인 o
## 3일 이후 wallpaper df에서 user id 있는 기기 id 추출
# df_prop_after = SpherePrep.Prop.df_prop_pipe(
#                 df_app_log_after, KEY_ID = [KEY_ID_DEVICE, KEY_ID_USER], drop_none_prop=False)  

lst_device_login_after = list(
                            df_app_log_wallpaper_after[KEY_ID_DEVICE][
                                df_app_log_wallpaper_after[KEY_ID_USER].apply(
                                    lambda x: True if pd.isna(x)==False else False)].unique())

lst_x1 = list(set(lst_y1) & set(lst_device_login_after))

print('x: ', len(lst_x1))
print('conversion: ', (len(lst_x1) / len(lst_y1)) * 100)

y:  2563
x:  380
conversion:  14.8263753413968


In [8]:
len(lst_device_login_past)
len(lst_device_loginx_past)
len(lst_wallpaper_after)
len(lst_device_login_after)

78615

2611013

2887

637

## 3)3차 수정
- 전체 미회원유저 중  신규 가입 유저 비율

In [13]:
#########################################
# 3. 전체 신규 회원가입율
#########################################
# 1) Y = 3일 이전 미회원가입 & 3일 이후 월페이퍼 접속 x 
## 3일 이전 미회원가입 = lst_device_loginx_past
## 3일 이후 월페이퍼 접속 x인 df_app_log 추출
## 3일 이후 전체 로그 데이터 = df_app_log2
df_app_log2 = df_app_log[df_app_log['date'] >= '20221103']
df_app_log2

,date,duration,sid,seq,app_key,sphere_id,user_id,uid,adid,gender,birth_y,user_grade,created_date,events,referrer,install_referrer,session,abs_events,day
2832592,2022-11-03 00:00:01,1.417,qNkvLZyE1MtA50azFbB4yV,1,qsJfZhE2Ft85SvZnBo2KSL,cqVsWeglzTk5oY-LdnY_2VS25tz0D,None,jB2WwGC2pyaFsLQ4fhi1wA,,,0,,2022-11-03 00:00:01,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Spider'}, 'ip': '211.249...",[sapContentDetail],2022-11-03
2852341,2022-11-03 00:00:02,2.029,A98pIjzIv-nVMKJOyFtL2o,1,qsJfZhE2Ft85SvZnBo2KSL,i5vczikmqeFOMjUUQFDeMluLD6_gy,None,zztT2PF7LcA_X5BJE8HDmg,,,0,,2022-11-03 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '211.36...",[sapContentDetail],2022-11-03
2815328,2022-11-03 00:00:02,5.476,3qH-2uS-XIxIpiwrN6PHWN,1,qsJfZhE2Ft85SvZnBo2KSL,r4rAOWX6H0V0azgWUMjTNV1ljJzOq,None,GXTjdpCTX3UN5gd4ujh_-g,,,0,,2022-11-03 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': ''}, 'ip': '58.231.198.16...",[sapContentDetail],2022-11-03
2850740,2022-11-03 00:00:03,4.213,1tPE_bQI3iDdyiKg5yoyxT,1,qsJfZhE2Ft85SvZnBo2KSL,uOS-S4A8f_8LKJvOFCwi48KsyBXWL,None,aA1uK29R_x26nTdHtXBqtg,,,0,,2022-05-29 19:13:27,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '118.23...",[sapContentDetail],2022-11-03
2802722,2022-11-03 00:00:06,0.000,T7JENW2TRyurPWMcwjoCgg,1,qsJfZhE2Ft85SvZoXInoo3,bBuXYQcESe5N9P_E1yzQUVfTrpoBh,None,6RIJbuQ2TpS3SVflUIxctA,af50c1e4-2030-4dd2-bd3a-200d0869169a,,0,,2022-11-03 00:00:06,"[{'name': '#appInstall', 'timestamp': 16674012...",[],[],"{'device': {'brand': 'samsung'}, 'ip': '110.70...",[#appInstall],2022-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56389,2022-11-30 23:59:54,1.023,-YE_P2uO9ewr3IURep5Hl6,1,qsJfZhE2Ft85SvZnBo2KSL,F4P_6aGW77usar63mD1yrwOwmqWID,None,XE58u16BuftapSgGocpfEg,,,0,,2022-11-30 23:59:54,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Spider'}, 'ip': '211.249...",[sapContentDetail],2022-11-30
57170,2022-11-30 23:59:54,0.000,bGw9OzrsFCrxpdMAwkawY2,1,qsJfZhE2Ft85SvZnBo2KSL,TiMTmgWCvOvf4eSLcKV-Yzhqo8TAK,None,GAjLwvw0oThP84Vy2-Yq-w,,,0,,2022-11-30 23:59:54,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Apple'}, 'ip': '180.70.6...",[sapContentDetail],2022-11-30
55745,2022-11-30 23:59:55,0.000,_nXXuC5ty4neNWQnC0ig8K,1,qsJfZhE2Ft85SvZnBo2KSL,_3kaod1Ip6DqmNNljHNcWp6yYHdmy,None,gHS5fMpazsFmi_usSqxTXw,,,0,,2022-11-30 23:59:55,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Samsung'}, 'ip': '223.38...",[sapContentDetail],2022-11-30
56484,2022-11-30 23:59:58,1.180,tLX7926KFOU2oyAYlvvO8q,1,qsJfZhE2Ft85SvZnBo2KSL,EqHSrhDaaBXvTR0z1DDgVrA4wu-WJ,None,z1JP4JN5z8DP7Vppr4-Vdg,,,0,,2022-10-19 12:18:41,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Samsung'}, 'ip': '219.25...",[sapContentDetail],2022-11-30


In [14]:
#########################################
# 3. 전체 신규 회원가입율
#########################################
# 1) Y = 3일 이전 미회원가입 & 3일 이후 월페이퍼 접속 x 
## 3일 이전 미회원가입 = lst_device_loginx_past
### 3일 이후 전체 로그 데이터 = df_app_log2
df_app_log_wallpaperx_after = df_app_log2[
                                df_app_log2['abs_events'].apply(
                                    lambda x: False if EVENT_MAIN_WALLPAPER in x else True)]
### 3일 이후 월페이퍼 접속하지 않은 로그 데이터 추출
lst_device_wallpaper_x_after = list(df_app_log_wallpaperx_after[KEY_ID_DEVICE].unique())


lst_y2 = list(
            set(lst_device_loginx_past) & set(lst_device_wallpaper_x_after)
        )

print('y: ', len(lst_y2))
############################################################################################
# 2) X = (3일 이전 미회원가입 & 3일 이후 월페이퍼 접속 x ) n (3일 이후 로그인)
## 3일 이후  로그인 o
lst_device_login_after_wx = list(
                            df_app_log_wallpaperx_after[KEY_ID_DEVICE][
                                df_app_log_wallpaperx_after[KEY_ID_USER].apply(
                                    lambda x: True if pd.isna(x)==False else False)].unique())

lst_x2 = list(set(lst_y2) & set(lst_device_login_after_wx))

print('x: ', len(lst_x2))
print('conversion: ', (len(lst_x2) / len(lst_y2)) * 100)

y:  1783987
x:  19936
conversion:  1.1174969324328037


In [24]:
len(lst_device_wallpaper_x_after)
len(lst_device_login_after_wx)

1801356

36404

In [16]:
df_app_log[KEY_ID_USER].isna().sum()
df_app_log2[KEY_ID_USER].isna().sum()

3925732

2767143

##### Original code -> have to debugging
- find a cause of error

In [33]:
#########################################
# 3. 전체 신규 회원가입율
#########################################
# 1) Y = 3일 이전 미회원가입 & 3일 이후 월페이퍼 접속 x 
## 3일 이전 미회원가입 = lst_device_loginx_past
## 3일 이후 월페이퍼 접속 x인 df_app_log 추출
df_wallpaperx_after_org = df_app_log_after[
                                df_app_log_after['abs_events'].apply(
                                    lambda x: False if EVENT_MAIN_WALLPAPER in x else True)]

lst_device_wallpaper_x_after_org = list(df_wallpaperx_after_org[KEY_ID_DEVICE].unique())


lst_y_org = list(
            set(lst_device_loginx_past) & set(lst_device_wallpaper_x_after_org)
        )

print('y: ', len(lst_y_org))
############################################################################################
# 2) X = (3일 이전 미회원가입 & 3일 이후 월페이퍼 접속 x ) n (3일 이후 로그인)
## 3일 이후  로그인 o
lst_device_login_after_wx_org = list(
                            df_wallpaperx_after_org[KEY_ID_DEVICE][
                                df_wallpaperx_after_org[KEY_ID_USER].apply(
                                    lambda x: True if pd.isna(x)==False else False)].unique())

lst_x_org = list(set(lst_y_org) & set(lst_device_login_after_wx_org))

print('x: ', len(lst_x_org))
print('conversion: ', (len(lst_x_org) / len(lst_y_org)) * 100)

y:  1783987
x:  1783987
conversion:  100.0


In [36]:
## Debugging
df_app_log_after
df_app_log_after_prep = SpherePrep.BasicPrep.basic_prep(df_app_log_after) 
df_app_log_after_prep

,date,duration,sid,seq,app_key,sphere_id,user_id,uid,adid,gender,birth_y,user_grade,created_date,events,referrer,install_referrer,session,abs_events
0,2022-11-30 00:10:09,33911,5q2vaG9mQzqOPMTmS-CECA,2,qsJfZhE2Ft85SvZoXInoo3,MFJCTqsYn7nz0Tcdd-cn4Mch4J_J2,,32Zwb7MfSPOHkL2ilAtoug,1d986fd3-3178-4b8e-bf4b-d61c2a9c9cac,,0,,2022-11-30 00:10:09,"[{'name': 'sapHome', 'timestamp': 166973462262...",[],"[{'atb_type': 'kochava', 'network': '', 'campa...","{'device': {'brand': 'samsung'}, 'ip': '180.65...","[sapHome, drawerMain]"
1,2022-11-30 00:04:20,389183,PlwsKehHQQaC4lD12ogMVQ,35,qsJfZhE2Ft85SvZoXInoo3,JpJpAJVqSv6gcec7wtF2u7xELtoWe,,9_hnO20nTIWH4fYP5R8Xrw,d6d0aceb-a049-448f-80b4-555adfe695a0,,0,,2022-09-11 00:21:11,"[{'name': 'sapHome', 'timestamp': 166973426284...",[],"[{'atb_type': 'kochava', 'network': '', 'campa...","{'device': {'brand': 'samsung'}, 'ip': '211.23...","[sapHome, sapContentDetail, sapContentDetail, ..."
2,2022-11-30 00:10:09,0,5q2vaG9mQzqOPMTmS-CECA,1,qsJfZhE2Ft85SvZoXInoo3,MFJCTqsYn7nz0Tcdd-cn4Mch4J_J2,,32Zwb7MfSPOHkL2ilAtoug,1d986fd3-3178-4b8e-bf4b-d61c2a9c9cac,,0,,2022-11-30 00:10:09,"[{'name': '#appInstall', 'timestamp': 16697346...",[],"[{'atb_type': 'kochava', 'network': '', 'campa...","{'device': {'brand': 'samsung'}, 'ip': '180.65...",[#appInstall]
3,2022-11-30 00:32:38,380993,DhYwbaFJRfG0qufX7DOlsg,38,qsJfZhE2Ft85SvZoXInoo3,JpJpAJVqSv6gcec7wtF2u7xELtoWe,,9_hnO20nTIWH4fYP5R8Xrw,d6d0aceb-a049-448f-80b4-555adfe695a0,,0,,2022-09-11 00:21:11,"[{'name': 'sapContentDetail', 'timestamp': 166...",[],"[{'atb_type': 'kochava', 'network': '', 'campa...","{'device': {'brand': 'samsung'}, 'ip': '211.23...","[sapContentDetail, sapContentDetail, sapConten..."
4,2022-11-30 00:39:03,0,BTkRRzV2TvmPY9fCmN7M7g,1,qsJfZhE2Ft85SvZoXInoo3,41IzzJy2LMD1NJwjXAREpzpebpW2h,,67QAvz81TTWBDxYePclDTA,d46c2fd1-356c-4d7e-9f00-48276287771d,,0,,2022-11-30 00:39:03,"[{'name': '#appInstall', 'timestamp': 16697363...",[],"[{'atb_type': 'kochava', 'network': '', 'campa...","{'device': {'brand': 'samsung'}, 'ip': '106.10...",[#appInstall]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862690,2022-11-03 23:21:42,148,egOFUhrb7GCSYXbCC786ra,1,qsJfZhE2Ft85SvZnBo2KSL,8TBtBbQ-dMChkjP9xZNULYH6q9IN2,,Sm5rfdlMHvvaRU-4j1OEug,,,0,,2022-11-03 23:21:42,"[{'name': 'courseTravelMain', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Apple'}, 'ip': '211.36.1...",[courseTravelMain]
2862691,2022-11-03 23:21:44,3137,iWCPgg-zPocL4Q4AgZbQtF,1,qsJfZhE2Ft85SvZnBo2KSL,kBoCu-fr4Pe_65eN6_tpLBX7s-8tK,,jA8i8xdvsLLWf9M7N4GBTQ,,,0,,2022-10-23 07:42:54,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '106.10...",[sapContentDetail]
2862692,2022-11-03 23:53:33,880556,k2idR3DPKiWe3-YhEuu24S,1,qsJfZhE2Ft85SvZnBo2KSL,0tg33ANYZEnfk3A24YHi5CqVpYHq9,,9QFk7xLtLLxCXvPSZJvSOA,,,0,,2022-09-26 15:52:45,"[{'name': 'sapHome', 'timestamp': 166748721503...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '121.18...","[sapHome, homeSearch, sapContentDetail, sapCon..."
2862693,2022-11-03 23:54:51,847751,-A-p7dOweaPKNhTd5IXri3,1,qsJfZhE2Ft85SvZnBo2KSL,74djuegpKa7Og8CzIXuCfdixC_Mga,,J3qQ8w2lzkqWlBNufvM2OA,,,0,,2022-10-01 02:51:50,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '211.20...","[sapContentDetail, sapContentDetail]"


,date,duration,sid,seq,app_key,sphere_id,user_id,uid,adid,gender,birth_y,user_grade,created_date,events,referrer,install_referrer,session,abs_events,day
2832592,2022-11-03 00:00:01,1.417,qNkvLZyE1MtA50azFbB4yV,1,qsJfZhE2Ft85SvZnBo2KSL,cqVsWeglzTk5oY-LdnY_2VS25tz0D,,jB2WwGC2pyaFsLQ4fhi1wA,,,0,,2022-11-03 00:00:01,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Spider'}, 'ip': '211.249...",[sapContentDetail],2022-11-03
2815328,2022-11-03 00:00:02,5.476,3qH-2uS-XIxIpiwrN6PHWN,1,qsJfZhE2Ft85SvZnBo2KSL,r4rAOWX6H0V0azgWUMjTNV1ljJzOq,,GXTjdpCTX3UN5gd4ujh_-g,,,0,,2022-11-03 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': ''}, 'ip': '58.231.198.16...",[sapContentDetail],2022-11-03
2852341,2022-11-03 00:00:02,2.029,A98pIjzIv-nVMKJOyFtL2o,1,qsJfZhE2Ft85SvZnBo2KSL,i5vczikmqeFOMjUUQFDeMluLD6_gy,,zztT2PF7LcA_X5BJE8HDmg,,,0,,2022-11-03 00:00:02,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '211.36...",[sapContentDetail],2022-11-03
2850740,2022-11-03 00:00:03,4.213,1tPE_bQI3iDdyiKg5yoyxT,1,qsJfZhE2Ft85SvZnBo2KSL,uOS-S4A8f_8LKJvOFCwi48KsyBXWL,,aA1uK29R_x26nTdHtXBqtg,,,0,,2022-05-29 19:13:27,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Samsung'}, 'ip': '118.23...",[sapContentDetail],2022-11-03
2802722,2022-11-03 00:00:06,0.000,T7JENW2TRyurPWMcwjoCgg,1,qsJfZhE2Ft85SvZoXInoo3,bBuXYQcESe5N9P_E1yzQUVfTrpoBh,,6RIJbuQ2TpS3SVflUIxctA,af50c1e4-2030-4dd2-bd3a-200d0869169a,,0,,2022-11-03 00:00:06,"[{'name': '#appInstall', 'timestamp': 16674012...",[],[],"{'device': {'brand': 'samsung'}, 'ip': '110.70...",[#appInstall],2022-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56389,2022-11-30 23:59:54,1.023,-YE_P2uO9ewr3IURep5Hl6,1,qsJfZhE2Ft85SvZnBo2KSL,F4P_6aGW77usar63mD1yrwOwmqWID,,XE58u16BuftapSgGocpfEg,,,0,,2022-11-30 23:59:54,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': None, 'utm_campaign': None, 'ut...",[],"{'device': {'brand': 'Spider'}, 'ip': '211.249...",[sapContentDetail],2022-11-30
57170,2022-11-30 23:59:54,0.000,bGw9OzrsFCrxpdMAwkawY2,1,qsJfZhE2Ft85SvZnBo2KSL,TiMTmgWCvOvf4eSLcKV-Yzhqo8TAK,,GAjLwvw0oThP84Vy2-Yq-w,,,0,,2022-11-30 23:59:54,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Apple'}, 'ip': '180.70.6...",[sapContentDetail],2022-11-30
55745,2022-11-30 23:59:55,0.000,_nXXuC5ty4neNWQnC0ig8K,1,qsJfZhE2Ft85SvZnBo2KSL,_3kaod1Ip6DqmNNljHNcWp6yYHdmy,,gHS5fMpazsFmi_usSqxTXw,,,0,,2022-11-30 23:59:55,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Samsung'}, 'ip': '223.38...",[sapContentDetail],2022-11-30
56484,2022-11-30 23:59:58,1.180,tLX7926KFOU2oyAYlvvO8q,1,qsJfZhE2Ft85SvZnBo2KSL,EqHSrhDaaBXvTR0z1DDgVrA4wu-WJ,,z1JP4JN5z8DP7Vppr4-Vdg,,,0,,2022-10-19 12:18:41,"[{'name': 'sapContentDetail', 'timestamp': 166...","{'utm_source': 'NAVER_GFA', 'utm_campaign': '2...",[],"{'device': {'brand': 'Samsung'}, 'ip': '219.25...",[sapContentDetail],2022-11-30


In [45]:
## conversion이 100%가 나오는 원인 파악함  --> 왜 user_id가 없는 애들이 없을까?
df_app_log_after[KEY_ID_USER].isna().sum()

df_app_log_after_prep[KEY_ID_USER].isna().sum()

df_app_log2[KEY_ID_USER].isna().sum()


print('-'*80)
df_app_log_after.shape
df_app_log_after_prep.shape
df_app_log2.shape

0

0

2767143

--------------------------------------------------------------------------------


(2862695, 18)

(2862695, 19)

(2862695, 19)

In [43]:
lst = []
df_list = []
_today = datetime.strptime('20221130', '%Y%m%d') + timedelta(days = 1) 
dates = (datetime.strptime('20221130', '%Y%m%d') - datetime.strptime('20221103', '%Y%m%d')).days + 1 

for i in range(1, dates+1):
    _d = _today - timedelta(days=i) # 설정한 today를 기준으로 과거 n일 json
    y = str(_d.year)
    m = str(_d.month).zfill(2)
    d = str(_d.day).zfill(2)
    _d = y+m+d
    lst.append(_d)
    _d

    df = pd.read_pickle(input_path + '/' + _d)
    df[KEY_ID_USER].isna().sum()
    df_list.append(df)

# df_total = pd.concat(df_list).reset_index(drop = True)

'20221130'

0

'20221129'

0

'20221128'

...

0

'20221104'

0

'20221103'

0